<a href="https://colab.research.google.com/github/kazimsayed954/News-Headline-Generator/blob/main/Notebook/News_Headline_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle (9).json to kaggle (9).json


{'kaggle (9).json': b'{"username":"sayedmohdkazimmehdi","key":"d4b8e57460316f6ca99b8cd91ec020bd"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Check the file in its new directory
!ls /root/.kaggle/
# Check the file permission
!ls -l ~/.kaggle/kaggle.json
#Change the file permission
# chmod 600 file – owner can read and write
# chmod 700 file – owner can read, write and execute
!chmod 600 ~/.kaggle/kaggle.json

kaggle.json
-rw-r--r-- 1 root root 75 Jan 22 03:35 /root/.kaggle/kaggle.json


In [6]:
!kaggle datasets download -d therohk/million-headlines -p /content/drive/MyDrive/Dataset/news\ headline

 47% 10.0M/21.1M [00:00<00:00, 101MB/s]
100% 21.1M/21.1M [00:00<00:00, 135MB/s]


In [7]:
!unzip /content/drive/MyDrive/Dataset/news\ headline/million-headlines.zip -d /content/drive/MyDrive/Dataset/news\ headline

Archive:  /content/drive/MyDrive/Dataset/news headline/million-headlines.zip
  inflating: /content/drive/MyDrive/Dataset/news headline/abcnews-date-text.csv  


In [2]:
!pip install textgenrnn

In [13]:
 from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model

In [17]:
import os
import pandas as pd
import numpy as np
from textgenrnn import textgenrnn
import time

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [18]:
filename = f"/content/drive/MyDrive/Dataset/news headline/abcnews-date-text.csv"
with open(os.path.join(".", "data", filename), "rb") as f:
    df = pd.read_csv(f)

In [19]:
NUM_EPOCHS = 1
PERCENT_TRAINING = 0.5

In [20]:
# exploratory data analysis
print(f"\nDataframe shape: {df.shape}\n") # 2 data points, 1226258 rows
print(f"First data entry: \n{df.head(1)}\n")
print(f"Sum of null values in dataset: {np.sum(df.isna().sum())}\n")  # there are null values
print(f"Number of unique values in the dataset: \n{df.nunique()}")
df = df.drop_duplicates(subset='headline_text')
df = df.sample(frac=PERCENT_TRAINING,random_state=0) 
print(f"\nDataframe shape after dropping duplicate headlines, and removing {(1 - PERCENT_TRAINING) * 100}% of data: {df.shape}\n")



Dataframe shape: (1226258, 2)

First data entry: 
   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...

Sum of null values in dataset: 0

Number of unique values in the dataset: 
publish_date        6517
headline_text    1195191
dtype: int64

Dataframe shape after dropping duplicate headlines, and removing 50.0% of data: (597596, 2)



In [21]:
arr = df["headline_text"].to_numpy()
model = textgenrnn()

In [22]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 7,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}
train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': NUM_EPOCHS,   # set higher to train the model for longer
    'gen_epochs': 2,   # generates sample text from model after given number of epochs
    'train_size': 1,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.2,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False  # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
model.train_on_texts(arr, 
    num_epochs=NUM_EPOCHS,
    new_model=True,
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=512,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=200,
    word_level=model_cfg['word_level'])

model.save(weights_path=f"/content/drive/MyDrive/Dataset/news headline/news_headline_model_{NUM_EPOCHS}_epochs_{PERCENT_TRAINING}.hdf5")

Training new model w/ 3-layer, 128-cell LSTMs


In [ ]:
temps = np.linspace(0.1,1,10)
model.generate_samples(n=20, temperatures=temps)
model.generate_samples(n=5, temperatures=temps, prefix="covid-19")